In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models

In [69]:
def prepereData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,
                          header=[0, 1, 2])
    generes = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right lables
    generes.columns = range(generes.shape[1])
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    generes[0] = generes[0] - 1
    data_y = [targets[i - 1] for i in generes[0]]
    generes[0] = data_y
    
    test = generes.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
    split.index = generes.index
    generes = split
    return data_x, generes

In [ ]:
data, labels = prepereData()
data, labels = 
x_train, x_rest, y_train, y_rest = train_test_split(data_x, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)

In [97]:
CNN = models.Sequential()
CNN.add(layers.InputLayer(input_shape=(x_train.shape[1],)))
CNN.add(layers.Conv1D(32, 5, activation='relu'))
CNN.add(layers.MaxPooling1D(4))
CNN.add(layers.Conv1D(64, 5, activation='relu'))
CNN.add(layers.MaxPooling1D(4))
CNN.add(layers.Conv1D(128, 5, activation='relu'))
CNN.add(layers.Dense(128, activation='relu'))
CNN.add(layers.Dense(64, activation='relu'))
CNN.add(layers.Dense(161))

ValueError: Input 0 of layer "conv1d_59" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 518)

In [94]:
CNN.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_56 (Conv1D)          (None, None, 32)          82912     
                                                                 
 max_pooling1d_33 (MaxPoolin  (None, None, 32)         0         
 g1D)                                                            
                                                                 
 conv1d_57 (Conv1D)          (None, None, 64)          10304     
                                                                 
 max_pooling1d_34 (MaxPoolin  (None, None, 64)         0         
 g1D)                                                            
                                                                 
 conv1d_58 (Conv1D)          (None, None, 128)         41088     
                                                                 
 dense_36 (Dense)            (None, None, 128)       

In [95]:
CNN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [96]:
training = CNN.fit(x_train,y_train,epochs=10, validation_data=(x_valid,y_valid))

Epoch 1/10


ValueError: in user code:

    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/bravo8234/anaconda3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 250, in assert_input_

In [72]:
x_train.shape

(39678, 518)

In [71]:
y_train.shape

(39678, 161)

In [67]:
y_train

statistics,Unnamed: 519_level_1
number,Unnamed: 519_level_2
track_id,
44204,11
16191,11
27388,14
59663,1
85101,31
...,...
34204,11
33552,20


<class 'pandas.core.frame.DataFrame'>
Int64Index: 39678 entries, 14416 to 126511
Columns: 518 entries, 0 to 517
dtypes: float64(518)
memory usage: 157.1 MB
